<a href="https://colab.research.google.com/github/Rogercaste/Bachelor_thesis/blob/main/detection_for_edf_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pyedflib
import numpy as np
import matplotlib.pyplot as plt
import statistics

file_name = 'Hippocampus_R.edf'

with pyedflib.EdfReader(file_name) as f:
    n = f.signals_in_file
    signal_labels = f.getSignalLabels()
    f_list= f.getSampleFrequencies()
    signals = np.zeros((n, f.getNSamples()[0]))
    for i in np.arange(n):
        signals[i, :] = f.readSignal(i)
print(signals.shape)
print(signals[0])

from ripple_detection import filter_ripple_band
from ripple_detection import Karlsson_ripple_detector, Kay_ripple_detector
for i in range(signals.shape[0]):
    SAMPLING_FREQUENCY = f_list[i]
    lfps = (signals[i])
    print(len(lfps))
    n_samples = lfps.shape[0]
    time = np.linspace(0, int(n_samples/SAMPLING_FREQUENCY), n_samples)
    speed = np.ones_like(time)
    print(lfps)
    lfps = [[x] for x in lfps]
    filtered_lfps = filter_ripple_band(np.array(lfps))
    Karlsson_ripple_times = Karlsson_ripple_detector(
        time, filtered_lfps, speed, SAMPLING_FREQUENCY
    )

    display(Karlsson_ripple_times)

    fig, ax = plt.subplots(figsize=(15, 3))
    ax.plot(time, lfps)

    #for midtime in true_ripple_midtime:
    #    ax.axvspan(
    #        midtime - RIPPLE_DURATION / 2,
    #        midtime + RIPPLE_DURATION / 2,
    #        alpha=0.3,
    #        color="green",
    #        zorder=9,
    #    )

    ripple_times=[]
    for ripple in Karlsson_ripple_times.itertuples():
        ax.axvspan(ripple.start_time, ripple.end_time, alpha=0.3, color="red", zorder=10)
        print(ripple.start_time)
        ripple_times.append(ripple.start_time)

    differences = []
    for i in range(len(ripple_times) - 1):
        diff = ripple_times[i+1] - ripple_times[i]
        differences.append(diff)
    print(differences)
    if len(differences)>1:
        print("std=%d"% statistics.stdev(differences))
